This code use all the dataset USDJPY M5 dataset from -2004.1.1-2020.6.9,if you want
to change it,just slice the dataframe in the data processing stage.
Meanwhile,in order to prevent the label is too small to affect the prediction, label has been multiplied by 1000 in data process stage.So when the final result is calculated, the value of the label will be divided by 1000

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pylab
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import torchvision.models as models
from torchvision import transforms 
from torch.autograd import Variable
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torchvision.transforms.functional as F
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 

Get processed dataset from the google drive,or you can upload the file directly to the colab

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



id = "Use the dataset from DataProcessing"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('DeLabeledUSDJPY-5M-2019.1.1-2020.6.9.csv')
df_JPYUSDM5=pd.read_csv('DeLabeledUSDJPY-5M-2019.1.1-2020.6.9.csv')

with pd.option_context('display.precision', 20):  
    dataM1 = pd.DataFrame(df_M1)

    dataM5=pd.DataFrame(df_JPYUSDM5)
    # show all the columns
    pd.set_option('display.max_columns', None)
    # show all the rows
    pd.set_option('display.max_rows', None)


In [ ]:
# Hypterparameters
days=30
BATCH_SIZE=64
LR=1e-3
epochs=100

Defining functions and data processing for feature extraction

In [ ]:
class TensorsDataset(torch.utils.data.Dataset):

    '''
    A simple loading dataset - loads the tensor that are passed in input. This is the same as
    torch.utils.data.TensorDataset except that you can add transformations to your data and target tensor.
    Target tensor can also be None, in which case it is not returned.
    '''

    def __init__(self, data_tensor, target_tensor=None, transforms=None, target_transforms=None):
        if target_tensor is not None:
            assert data_tensor.size(0) == target_tensor.size(0)
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor

        if transforms is None:
            transforms = []
        if target_transforms is None:
            target_transforms = []

        if not isinstance(transforms, list):
            transforms = [transforms]
        if not isinstance(target_transforms, list):
            target_transforms = [target_transforms]

        self.transforms = transforms
        self.target_transforms = target_transforms

    def __getitem__(self, index):

        data_tensor = self.data_tensor[index]
        for transform in self.transforms:
            data_tensor = transform(data_tensor)

        if self.target_tensor is None:
            return data_tensor

        target_tensor = self.target_tensor[index]
        for transform in self.target_transforms:
            target_tensor = transform(target_tensor)

        return data_tensor, target_tensor

    def __len__(self):
        return self.data_tensor.size(0)

In [ ]:
def mean_absolute_percentage_error(y_pred, y_true):
    mask=y_true!=0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / (y_true[mask]))) * 100

def normalize_function(data):
    min = np.amin(data,axis=0)
    max = np.amax(data,axis=0)
    return (data - min)/(max-min)
def labeled(x):
  if x>=0:
    return 0
  if x<=-0:
    return 1  

def DA(predictions,y_test):
    result=predictions*y_test
    result=[1 if item>=0 else 0 for item in result]
    return np.mean(result)

def DA1(true,predict,yesday):
    result=(true-yesday)*(predict-yesday)
    result=[1 if item>=0 else 0 for item in result]
    return np.mean(result)

In [ ]:
IndicatorValueM5=dataM5[['Close_wave','Open_wave','High_wave','Low_wave','RSI5','RSI10','RSI20','macd','macd','macdsignal','slowk','slowd','fastk','fastd','WR5','WR10', 'WR20','ROC5','ROC10','ROC20','CCI5','CCI10','CCI20','ATR5','ATR10','ATR20','NATR5','NATR10','NATR20','TRANGE']].to_numpy()
IndicatorValueM5=IndicatorValueM5[33:]
# nomalize
IndicatorValueM5=normalize_function(IndicatorValueM5)[:-4]
print(IndicatorValueM5.shape)
IndicatorValueM5=np.array([IndicatorValueM5[i-days:i] for i in range(days,len(IndicatorValueM5))])
print(IndicatorValueM5.shape)

labelM5=dataM5.label4[days:].to_numpy()
labelM5=labelM5[33:-4]
print(labelM5.shape)



IndicatorValueM5=torch.tensor(IndicatorValueM5)
IndicatorValueM5 = torch.unsqueeze(IndicatorValueM5, 1)
labelM5=torch.tensor(labelM5)
print(IndicatorValueM5.shape)

# Cureently FinalM5_train use all the dataset to extract the feature,if you want to train, test or verification the model 
# remove the following annotations or change it to any length you want

# FinalM5_train=IndicatorValueM5[:99000]
FinalM5_train=IndicatorValueM5
FinalM5_test=IndicatorValueM5[99000:]

# labelM5_train=labelM5[:99000]
labelM5_train=labelM5
labelM5_test=labelM5[99000:]

transformer = transforms.Compose([
  # transforms.Resize((224,224)),
  # transforms.ToPILImage() ,
  # transforms.ToTensor(), 
   
])

In [ ]:
FinaldatasetM5_train = TensorsDataset(FinalM5_train, labelM5_train,transformer)
FinaldatasetM5_test = TensorsDataset(FinalM5_test, labelM5_test,transformer)
FinaldatasetM5_FE=TensorsDataset(IndicatorValueM5, labelM5,transformer)

data_loader_FinalM5train = DataLoader(FinaldatasetM5_train, batch_size=BATCH_SIZE,shuffle=True)
data_loader_FinalM5test = DataLoader(FinaldatasetM5_test, batch_size=BATCH_SIZE,shuffle=False)
data_loader_FinalM5FE = DataLoader(FinaldatasetM5_FE, batch_size=BATCH_SIZE,shuffle=False)

Define ResNet structure

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes,
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = torch.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1024):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv0 = nn.Sequential(         
            nn.Conv2d(1, 3, 1,1,0), 
            nn.BatchNorm2d(3,affine=True), 
              nn.ELU(),                      
        )

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

        self.bn = nn.BatchNorm1d(1024, momentum=0.9)
        self.linear2= nn.Linear(1024,256)
        self.bn2 = nn.BatchNorm1d(256, momentum=0.9)
        self.linear3= nn.Linear(256,1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv0(x)
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.nn.functional.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        
        output=self.bn(out)
        output = torch.relu(output)
        output=self.linear2(output)
        output=self.bn2(output)
        output = torch.relu(output)
        output=self.linear3(output)
        return output,out


In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(1, 3, 1,1,0), 
            nn.BatchNorm2d(3,affine=True), 
              nn.ELU(),                      
        )
        self.net= models.resnext50_32x4d(pretrained=False)
        # self.net=models.wide_resnet50_2(pretrained=True)
       
        
        self.net._modules["fc"] = nn.Linear(2048,1024)      
        self.linear1 = nn.Linear(1024,1024)
        self.bn1 = nn.BatchNorm1d(1024, momentum=0.9)
        self.linear2= nn.Linear(1024,256)
        self.bn2 = nn.BatchNorm1d(256, momentum=0.9)
        self.linear3= nn.Linear(256,1)




    def forward(self, x):
      
            x = self.conv1(x)        
            x = self.net(x)
            x = self.linear1(x)
            x = self.bn1(x)
            x = torch.relu(x)
            output=self.linear2(x)
            output=self.bn2(output)
            output = torch.relu(output)
            output=self.linear3(output)
            
  
            return output,x

Currently,this code use all the dataset to extract the feature,if you want to retrain the ResNet using the train,test dataset,remove the following test and verification annotations(Don't forget to change the current train dataset and code,it depends on how large datasets you want to use and how to allocate train test sets).These codes will also save the best model automatically and extract the features in the next part.

In [ ]:
# Hypterparameters
days=30
BATCH_SIZE=32
LR=1e-3
epochs=100

# Mynet=ResNet(Bottleneck, [3,4,6,3])
Mynet=MyModel()
Mynet = Mynet.double()

use_gpu = torch.cuda.is_available()

if use_gpu:
    print("using gpu")
    Mynet = Mynet.cuda()
optimizer = torch.optim.Adam(Mynet.parameters(), lr=LR)

criterion = nn.MSELoss()

temp_loss=100000
for epoch in range(epochs):
  #Training phase
  total_training_loss = 0.0
  i=0
  pre=torch.zeros(1,1).double()
  label=torch.zeros(1,1).double()
  ###
  feature=torch.zeros(1,1024).double()
  ###
  for iter, traindata in enumerate(data_loader_FinalM5train):
    i+=1
    train_inputs, train_labels = traindata
   
    
    if use_gpu:    
      train_inputs, train_labels = train_inputs.cuda(), train_labels.cuda()      
    else:
      train_inputs, train_labels = train_inputs, train_labels     
   

    train_labels=train_labels.view(-1,1)
    # train_labels=torch.unsqueeze(train_labels, 1)


    train_outputs,feature1 = Mynet(train_inputs.double())
    

    loss = criterion(train_outputs, train_labels)
    optimizer.zero_grad()
    # torch.set_default_tensor_type(torch.FloatTensor)
    loss.backward()
    optimizer.step()
      
    total_training_loss += loss.data

    pre=torch.cat((pre.cuda(),train_outputs.cuda()), 0)
    label=torch.cat((label.cuda(),train_labels.cuda()), 0)
    feature=torch.cat((feature.cuda(),feature1.cuda()), 0)


    MSE=mean_squared_error(train_outputs.cpu().detach().numpy()/1000,train_labels.cpu().detach().numpy()/1000)
    MAE=mean_absolute_error(train_outputs.cpu().detach().numpy()/1000,train_labels.cpu().detach().numpy()/1000)
    RMSE=MSE ** 0.5
    MAPE=mean_absolute_percentage_error(train_outputs.cpu().detach().numpy()/1000,train_labels.cpu().detach().numpy()/1000)

    # print("OUTPUT",train_outputs[:10])
    # print("LABEL",train_labels[:10])
    if (iter + 1) %100 == 0:
      print('Training Phase: Epoch: [%2d][%2d/%2d]\tIteration Loss: %.6f\t' %
                (iter+1, epoch+1, epochs, loss.data ))#loss.data / train_labels.size(0)
      print('MAE %.6f*10-3\t MSE: %.6f*10-3\tRMSE: %.6f*10-3\tMAPE: %.6f%%\t' %
                (1000*MAE,1000*MSE,1000*RMSE,MAPE ))
  print("EPOCH: %2d\t Total Training Loss：%.6f\t "%
                ( epoch+1, total_training_loss/i))
  pre=pre[1:]
  label=label[1:] 
  feature=feature.cpu().detach().numpy()
  feature=feature[1:]
  MSE=mean_squared_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
  MAE=mean_absolute_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
  RMSE=MSE ** 0.5
  MAPE=mean_absolute_percentage_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
  print('TOTAL TRAIN:MAE %.6f*10-3\t MSE: %.6f*10-3\tRMSE: %.6f*10-3\tMAPE: %.6f%%\t' %
                (1000*MAE,1000*MSE,1000*RMSE,MAPE ))
  # print("PREDICTION:",pre[:10])
  # print("LABEL:",label[:10])

  ####
  # torch.save(Mynet, 'Mynet.pkl')
  ####




  #TEST Phase
#   i=0
#   total_test_loss=0.0
#   pre=torch.zeros(1,1).double()
#   label=torch.zeros(1,1).double()
#   for iter, testdata in enumerate(data_loader_FinalM5test):
#     i+=1
#     test_inputs, test_labels = testdata
#     if use_gpu:
#       test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
#     else:
#       test_inputs, test_labels = test_inputs, test_labels
#     test_labels=test_labels.view(-1,1)
#     test_outputs = Mynet(test_inputs.double())[0]
#     test_loss = criterion(test_outputs, test_labels)
#     total_test_loss += test_loss.data
#     pre=torch.cat((pre.cuda(),test_outputs.cuda()), 0)
#     label=torch.cat((label.cuda(),test_labels.cuda()), 0)
#   print("EPOCH: %2d\tTotal TEST Loss：%.6f\t "%
#                 (epoch+1,total_test_loss/i))
#   pre=pre[1:]
#   label=label[1:]
#   acc=DA(pre.cpu().detach().numpy(),label.cpu().detach().numpy())
#   MSE=mean_squared_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
#   MAE=mean_absolute_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
#   RMSE=MSE ** 0.5
#   MAPE=mean_absolute_percentage_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
#   print('TOTAL TEST:MAE %.6f*10-3\t MSE: %.6f*10-3\tRMSE: %.6f*10-3\tMAPE: %.6f%%\tDA: %.6f%%\t' %
#                 (1000*MAE,1000*MSE,1000*RMSE,MAPE,acc ))
#   print("PREDICTION:",pre[:10])
#   print("LABEL:",label[:10])
  
#   print("########################################\n########################################")
#   testloss=total_test_loss/i
#   if testloss<temp_loss:
#     temp_loss=testloss
#     lowest_epo=epoch
#     lowest_MAE=MAE
#     lowest_MSE=MSE
#     lowest_RMSE=RMSE
#     lowest_MAPE=MAPE
#     lowest_acc=acc
#     torch.save(Mynet, 'Mynet.pkl') 

# print("LOWEST TEST LOSS:%.6f\t in the epoch:%2d\t"%(temp_loss,lowest_epo+1))
# print('In this epoch :MAE %.6f*10-3\t MSE: %.6f*10-3\tRMSE: %.6f*10-3\tMAPE: %.6f%%\tACC: %.6f%%\t' %
#                 (1000*lowest_MAE,1000*lowest_MSE,1000*lowest_RMSE,lowest_MAPE,lowest_acc ))

In [ ]:
# # recover from the stored network

# # verification



# use_gpu = torch.cuda.is_available()
# recoverNet = torch.load('Mynet.pkl') 
# total_test_loss=0.0
# pre=torch.zeros(1,1).double()
# label=torch.zeros(1,1).double()
# criterion = nn.MSELoss()
# i=0
# for iter, testdata in enumerate(data_loader_FinalM5test):
#   i+=1
#   test_inputs, test_labels = testdata
#   if use_gpu:
#     test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
#   else:
#     test_inputs, test_labels = test_inputs, test_labels
#   test_labels=test_labels.view(-1,1)
#   test_outputs = recoverNet(test_inputs.double())[0]
#   test_loss = criterion(test_outputs, test_labels)
#   total_test_loss += test_loss.data
#   pre=torch.cat((pre.cuda(),test_outputs.cuda()), 0)
#   label=torch.cat((label.cuda(),test_labels.cuda()), 0)
# print("Recover Loss：%.6f\t "%
#                 (total_test_loss/i))
# pre=pre[1:]
# label=label[1:] 
# MSE=mean_squared_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
# MAE=mean_absolute_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
# RMSE=MSE ** 0.5
# MAPE=mean_absolute_percentage_error(pre.cpu().detach().numpy()/1000,label.cpu().detach().numpy()/1000)
# acc=DA(pre.cpu().detach().numpy(),label.cpu().detach().numpy())
# print('Recover TEST:MAE %.6f*10-3\t MSE: %.6f*10-3\tRMSE: %.6f*10-3\tMAPE: %.6f%%\DA: %.6f\t' %
#                 (1000*MAE,1000*MSE,1000*RMSE,MAPE,acc))


# # ectract feature and get prediction
# feature=torch.zeros(1,1024).double()
# prediction=torch.zeros(1,1).double()
# for iter, testdata in enumerate(data_loader_FinalM5FE):
#   test_inputs, test_labels = testdata
#   if use_gpu:
#     test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
#   else:
#     test_inputs, test_labels = test_inputs, test_labels
#   test_labels=test_labels.view(-1,1)
#   with torch.no_grad():
#     torch.cuda.empty_cache()
#     test_outputs,feature1 = recoverNet(test_inputs.double())
#     feature=torch.cat((feature.cuda(),feature1.cuda()), 0)
#     prediction=torch.cat((prediction.cuda(),test_outputs.cuda()), 0)

# prediction=prediction[1:]
# feature=feature[1:]
# feature=feature.cpu().detach().numpy()
# print(feature.shape)


# prediction=prediction.cpu().detach().numpy()
# print(prediction.shape)



Using LightGBM to deal with the extracted features from ResNet and the technical indicators.

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
# x=a.reshape((-1,days*a.shape[-1]))
x1=feature
print(x1.shape)
x2=dataM5[['RSI5','RSI10','RSI20','macd','macd','macdsignal','slowk','slowd','fastk','fastd','WR5','WR10', 'WR20','ROC5','ROC10','ROC20','CCI5','CCI10','CCI20','ATR5','ATR10','ATR20','NATR5','NATR10','NATR20','TRANGE']].to_numpy()
x2=x2[33:-4]
x2=np.array([x2[i-days:i] for i in range(days,len(x2))])
print(x2.shape)
x2=x2.reshape((-1,days*x2.shape[-1]))
print(x2.shape)
x=np.concatenate([x2, x1], 1)

y=dataM5.label4[33:].to_numpy()
y=y[days:-4]


print(x.shape)
print(y.shape)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=33) 
params = {'num_leaves': 38,              
          'min_data_in_leaf': 50,
          'objective': 'regression',     
          'max_depth': -1,                
          'learning_rate': 0.1,           
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",             
          "feature_fraction": 0.9,         
          "bagging_freq": 1,               
          "bagging_fraction": 0.7,          
          "bagging_seed": 11,
          "lambda_l1": 0.1,
          "verbosity": -1,               
          "nthread": 4,               
          'metric': 'mae',               
          "random_state": 2019,
          # 'device': 'gpu'
          }
trn_data = lgb.Dataset(x_train, y_train)
val_data = lgb.Dataset(x_test, y_test)

clf = lgb.train(params,
                trn_data,
                20000,
                valid_sets=[trn_data, val_data],
                verbose_eval=200,
                early_stopping_rounds=500)
oof = clf.predict(x_train, num_iteration=clf.best_iteration)
predictions = clf.predict(x_test, num_iteration=clf.best_iteration)


MSE=mean_squared_error(predictions/1000,y_test/1000)
MAE=mean_absolute_error(predictions/1000,y_test/1000)
RMSE=MSE ** 0.5
MAPE=mean_absolute_percentage_error(predictions/1000,y_test/1000)
acc=DA(predictions,y_test)

print('MAE %.6f*10-3\t MSE: %.6f*10-3\tRMSE: %.6f*10-3\tMAPE: %.6f%%\tACC: %.6f' %
                (1000*MAE,1000*MSE,1000*RMSE,MAPE,acc ))

In [ ]:
predictions1 = clf.predict(x, num_iteration=clf.best_iteration)
dataM5['pre_Value'] = ''

Save the results to the Google Drive

In [ ]:
for i in range(0,predictions1.shape[0]):
    dataM5['pre_Value'][i+63]=predictions1[i]

dataM5.to_csv('D1_Predicted.csv')
# Save file to google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'D1_Predicted.csv'})
uploaded.SetContentFile('D1_Predicted.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))